# VacationPy

In [9]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key= gkey)

 ## Store Part I results into DataFrame
 - Load the csv exported in Part I to a DataFrame

In [10]:
weather_df = pd.read_csv('../Resources/weather_df.csv')
weather_df

,Unnamed: 0,City,Cloudiness,Country,Humidity,Latitude,Longitude,Max Temperature,Wind Speed
0,0,Albany,100,US,47,42.60,-73.97,51.01,1.99
1,1,Ushuaia,75,AR,29,-54.80,-68.30,66.20,10.29
2,2,Rikitea,3,PF,63,-23.12,-134.97,72.97,3.76
3,3,Bethel,1,US,44,41.37,-73.41,60.80,5.82
4,4,Barrow,1,US,85,71.29,-156.79,10.40,12.75
...,...,...,...,...,...,...,...,...,...
555,555,Alofi,100,NU,100,-19.06,-169.92,75.20,4.70
556,556,Kostino,100,RU,94,58.91,53.27,19.81,7.76
557,557,Lębork,40,PL,72,54.54,17.75,41.00,21.92
558,558,Trowbridge,100,GB,90,51.32,-2.21,53.01,1.01


## Humidity Heatmap
- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.
- Create a heat map that displays the humidity for every city from the part I of the homework.

In [11]:
# Grab locations and humidity from weather_df
coord = weather_df[["Latitude", "Longitude"]]
hum = weather_df["Humidity"]


fig = gmaps.figure(zoom_level= 3, center= (35, -20))
heat_layer = gmaps.heatmap_layer(coord, weights= hum, dissipating= False, max_intensity= 100,
                                 point_radius= 1)


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

Narrow down the DataFrame to find your ideal weather condition:
- A max temperature lower than 80 degrees but higher than 70.
- Wind speed less than 10 mph.
- Zero cloudiness.


In [12]:
# Set the criteria for the ideal vacation weather condition

best_conditions_df = pd.DataFrame(weather_df, columns = ["City", 
                                                         "Country", 
                                                         "Latitude", 
                                                         "Longitude", 
                                                         "Max Temperature", 
                                                         "Wind Speed", 
                                                         "Cloudiness"])

good_max_temp = (best_conditions_df["Max Temperature"] <= 80) & (best_conditions_df["Max Temperature"] > 70)
good_wind_spd = best_conditions_df["Wind Speed"] < 10
good_cloudiness = best_conditions_df["Cloudiness"] == 0

hotel_df = best_conditions_df[good_max_temp & good_wind_spd & good_cloudiness]


hotel_df["Hotel Name"] = " "


## Hotel Map
Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.  
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
 

In [13]:
# Loop through hotel_df with api calls
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": gkey,
        }


        lat = row["Latitude"]
        lng = row["Longitude"]

        params["location"] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
      
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Hotel Name
35,Saint-Philippe,RE,-21.36,55.77,75.20,3.36,0,Les Embruns Du Baril
39,Puerto del Rosario,ES,28.50,-13.86,73.40,8.05,0,Hotel El Mirador
46,Antalaha,MG,-14.90,50.28,73.78,3.00,0,Villa-Malaza
147,Sitrah,BH,26.15,50.62,78.80,3.36,0,Al Bander Resort - منتجع البندر
177,Arlit,NE,18.74,7.39,75.07,7.94,0,Hôtel Telwa Bungalow
178,Riyadh,SA,24.69,46.72,75.20,5.82,0,voco Riyadh
200,Valparaíso,CL,-33.04,-71.63,75.20,8.05,0,Hotel Brighton
213,Los Llanos de Aridane,ES,28.66,-17.92,75.20,9.17,0,Hotel Hacienda de Abajo
339,Mount Isa,AU,-20.73,139.50,71.60,5.82,0,ibis Styles Mt Isa Verona
345,Río Gallegos,AR,-51.62,-69.22,71.60,5.82,0,Hotel Aire de Patagonia


In [14]:
# Drop the ones with NaN in Hotel Name column
hotel_df = hotel_df[hotel_df["Hotel Name"] != "NaN"]
hotel_df

,City,Country,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Hotel Name
35,Saint-Philippe,RE,-21.36,55.77,75.20,3.36,0,Les Embruns Du Baril
39,Puerto del Rosario,ES,28.50,-13.86,73.40,8.05,0,Hotel El Mirador
46,Antalaha,MG,-14.90,50.28,73.78,3.00,0,Villa-Malaza
147,Sitrah,BH,26.15,50.62,78.80,3.36,0,Al Bander Resort - منتجع البندر
177,Arlit,NE,18.74,7.39,75.07,7.94,0,Hôtel Telwa Bungalow
178,Riyadh,SA,24.69,46.72,75.20,5.82,0,voco Riyadh
200,Valparaíso,CL,-33.04,-71.63,75.20,8.05,0,Hotel Brighton
213,Los Llanos de Aridane,ES,28.66,-17.92,75.20,9.17,0,Hotel Hacienda de Abajo
339,Mount Isa,AU,-20.73,139.50,71.60,5.82,0,ibis Styles Mt Isa Verona
345,Río Gallegos,AR,-51.62,-69.22,71.60,5.82,0,Hotel Aire de Patagonia


- Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

In [15]:
# Grabbing from stater_code
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""


# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:

# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))